In [2]:
import numpy as np
from toml import load as tomlload
from alma import (read_model_pp, infer_rheology_pp, 
                  build_model, love_numbers)

ModuleNotFoundError: No module named 'joblib'

In [ ]:
alma_params = tomlload('paramsPyALMA3.toml')

# Check none for parallel
if type(alma_params['parallel']) is not bool:
    alma_params['parallel'] = None

alma_params

{'filename': 'EuropaProfile_Seawater_35.2ppt_Tb268.305K_red.txt',
 'rheology_structure': ['elastic', 'newton', 'newton', 'maxwell'],
 'layer_radius': False,
 'layer_radius_index': False,
 'num_digits': 128,
 'gorder': 8,
 'tau': 0,
 'mode': 'tidal',
 'dmin': 1,
 'dmax': 10,
 'dstep': 1,
 'time': 'log',
 'tpoints': 20,
 'timestart': -3,
 'timeend': 2,
 'function': 'step',
 'function_length': 1.0,
 'verbose': True,
 'type': 'real',
 'parallel': None}

In [ ]:
# Read PP model
model = read_model_pp(alma_params['filename'])

In [ ]:
rheology, params = infer_rheology_pp(model,
                                         structure=alma_params['rheology_structure'],
                                         layer_radius=alma_params['layer_radius'],
                                         layer_radius_index=alma_params['layer_radius_index'])

  >> Verifying validity of structure rheologies
  >> Found 4 pre-defined layers in model radius


In [ ]:
model_params = build_model(model['model'][:, model['columns'].index('r')],
                               model['model'][:, model['columns'].index('rho')],
                               model['mu'],
                               model['vis'],
                               rheology,
                               params,
                               ndigits = alma_params['num_digits'],
                               verbose = alma_params['verbose'])

> Initializing
  >> Setting precision: 128
> Building model
  >> Parsing Rheology
  >> Computing mass of the planet
  >> Computing gravity at the interface boundaries
  >> Normalizing model parameters


In [ ]:
# Harmonic degrees
n = np.arange(alma_params['dmin'], alma_params['dmax'], alma_params['dstep'])

In [ ]:
# Time step
if alma_params['time'] == 'lin':
    t = np.linspace(alma_params['timestart'], 
                 alma_params['timeend'], 
                 alma_params['tpoints'],
                 endpoint=True)
elif alma_params['time'] == 'log':
    t = np.logspace(alma_params['timestart'], 
                 alma_params['timeend'], 
                 alma_params['tpoints'],
                 endpoint=True)
elif alma_params['time'] == 'geom':
    t = np.geomspace(alma_params['timestart'], 
                 alma_params['timeend'], 
                 alma_params['tpoints'],
                 endpoint=True)
else:
    raise ValueError(f'ERROR: Unknown scale for time: {alma_params["time"]}.' +
                      'Allowed values are: "log", "geom", "lin".')

In [ ]:
# Compute love numbers
h, l, k = love_numbers(n, t, 
                       alma_params['mode'], 
                       alma_params['function'],
                       alma_params['tau'],
                       model_params,
                       alma_params['type'],
                       alma_params['gorder'],
                       verbose = alma_params['verbose'],
                       parallel = alma_params['parallel'])

> Computing Salzer weights
> Computing Love Numbers - Parallel


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:  4.9min remaining: 17.2min
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:  5.2min remaining:  2.6min


  >> Clearing spawned threads.


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  5.2min finished
